In [1]:
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d

In [2]:
def sounding_interp(snd_temp, snd_height, target_temp):
    """
    Provides an linear interpolated height for a target temperature using a
    sounding vertical profile. Looks for first instance of temperature
    below target_temp from surface upward.

    Parameters
    ----------
    snd_temp : ndarray
        Temperature data (degrees C).
    snd_height : ndarray
        Relative height data (m).
    target_temp : float
        Target temperature to find height at (m).

    Returns
    -------
    intp_h: float
        Interpolated height of target_temp (m).

    """
    intp_h = np.nan

    #check if target_temp is warmer than lowest level in sounding
    if target_temp>snd_temp[0]:
        print('warning, target temp level below sounding, returning ground level (0m)')
        return 0.
    
    
    # find index above and below freezing level
    mask = np.where(snd_temp < target_temp)
    above_ind = mask[0][0]

    # index below
    below_ind = above_ind - 1
    # apply linear interplation to points above and below target_temp
    set_interp = interp1d(
        snd_temp[below_ind:above_ind+1],
        snd_height[below_ind:above_ind+1], kind='linear')
    # apply interpolant
    intp_h = set_interp(target_temp)
    return intp_h


In [3]:
#config
access_root = '/g/data/lb4/ops_aps2/access-g/0001' #access g
request_date = '20200720' #yyyymmdd UTC
request_time = '0600' #HHMM UTC
request_lat = -27.717
request_lon = 153.24
request_name = '20200720_hailers'

In [4]:
#build file paths
access_folder = '/'.join([access_root, request_date, request_time, 'an', 'pl'])
temp_ffn = access_folder + '/air_temp.nc'
geop_ffn = access_folder + '/geop_ht.nc'
relh_ffn = access_folder + '/relhum.nc'
uwnd_ffn = access_folder + '/wnd_ucmp.nc'
vwnd_ffn = access_folder + '/wnd_vcmp.nc'

In [5]:
#extract data
temp_ds = xr.open_dataset(temp_ffn)
temp_data = temp_ds.air_temp.sel(lon=request_lon, method='nearest').sel(lat=request_lat, method='nearest').data[0] - 273.15 #units: deg C
geop_ds = xr.open_dataset(geop_ffn)
geop_data = geop_ds.geop_ht.sel(lon=request_lon, method='nearest').sel(lat=request_lat, method='nearest').data[0] #units: m
relh_ds = xr.open_dataset(relh_ffn)
relh_data = relh_ds.relhum.sel(lon=request_lon, method='nearest').sel(lat=request_lat, method='nearest').data[0] #units: percentage
uwnd_ds = xr.open_dataset(uwnd_ffn)
uwnd_data = uwnd_ds.wnd_ucmp.sel(lon=request_lon, method='nearest').sel(lat=request_lat, method='nearest').data[0] #units: m/s
vwnd_ds = xr.open_dataset(vwnd_ffn)
vwnd_data = vwnd_ds.wnd_vcmp.sel(lon=request_lon, method='nearest').sel(lat=request_lat, method='nearest').data[0] #units: m/s

In [6]:
#interpolate to 0C and -20C levels
print('0C altitude: ', np.round(sounding_interp(temp_data, geop_data, 0)), 'm')
print('-20C altitude: ', np.round(sounding_interp(temp_data, geop_data, -20)), 'm')

0C altitude:  4404.0 m
-20C altitude:  7361.0 m


In [ ]:
#output to file

